In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df=pd.read_csv("train_5.csv")

In [4]:
df = df.fillna(method="ffill")
pd.isnull(df).any()

row_id        False
time          False
x             False
y             False
direction     False
congestion    False
dtype: bool

In [5]:
df

,row_id,time,x,y,direction,congestion
0,0,1991-04-01 00:00:00,0,0,EB,70
1,1,1991-04-01 00:00:00,0,0,NB,49
2,2,1991-04-01 00:00:00,0,0,SB,24
3,3,1991-04-01 00:00:00,0,1,EB,18
4,4,1991-04-01 00:00:00,0,1,NB,60
...,...,...,...,...,...,...
848830,848830,1991-09-30 11:40:00,2,3,NB,54
848831,848831,1991-09-30 11:40:00,2,3,NE,28
848832,848832,1991-09-30 11:40:00,2,3,SB,68
848833,848833,1991-09-30 11:40:00,2,3,SW,17


In [9]:
del df['row_id']

In [10]:
df.columns

Index(['time', 'x', 'y', 'direction', 'congestion'], dtype='object')

### Feature enigneering

In [12]:
df.time = pd.to_datetime(df.time)

In [15]:
months = df.time.dt.month

In [17]:
day = df.time.dt.day

In [18]:
hours = df.time.dt.hour

In [19]:
to_one_hot = df.time.dt.day_name()

days = pd.get_dummies(to_one_hot)

In [22]:
def daypart(hour):
    if hour in [2,3,4,5]:
        return "dawn"
    elif hour in [6,7,8,9]:
        return "morning"
    elif hour in [10,11,12,13]:
        return "noon"
    elif hour in [14,15,16,17]:
        return "afternoon"
    elif hour in [18,19,20,21]:
        return "evening"
    else: return "midnight"
    
raw_dayparts = hours.apply(daypart)

dayparts = pd.get_dummies(raw_dayparts)

dayparts = dayparts[['dawn','morning','noon','afternoon','evening','midnight']]

In [24]:
day_names = df.time.dt.day_name()
is_weekend = day_names.apply(lambda x : 1 if x in ['Saturday','Sunday'] else 0)

In [83]:
X= pd.DataFrame({
    'x': df.x , 'y':df.y , 'direction':df.direction,
    'month' : months,
    'day' : day,
    'hour' : hours,
    'is_weekend': is_weekend

})

### EDA

##### Heat map

In [ ]:
plt.figure(figsize=(18,10))
plt.scatter(X['day'],X['hour'],c=df['congestion'], cmap='cool', alpha = 0.8)
plt.colorbar().set_label("COngestion Statistics")
plt.title('Day of the week')
plt.xlabel('Hour of the day')
plt.ylabel('Congestion levels')
plt.show()

##### Traffic Direction

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(hd.ocean_proximity, bins = 5, ec = 'black', color = 'violet')
plt.xlabel('Direction', fontsize=16)
plt.ylabel('Amt of traffic', fontsize=16)
plt.title('Teaffic diresction', fontsize=16)

### Features and variables

In [84]:
y = df.iloc[:, -1]

In [85]:
X

,x,y,direction,month,day,hour,is_weekend
0,0,0,EB,4,1,0,0
1,0,0,NB,4,1,0,0
2,0,0,SB,4,1,0,0
3,0,1,EB,4,1,0,0
4,0,1,NB,4,1,0,0
...,...,...,...,...,...,...,...
848830,2,3,NB,9,30,11,0
848831,2,3,NE,9,30,11,0
848832,2,3,SB,9,30,11,0
848833,2,3,SW,9,30,11,0


### Encoding

In [86]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [-5])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [92]:
X

array([[ 1.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  1.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ..., 30., 11.,  0.],
       [ 0.,  0.,  0., ..., 30., 11.,  0.],
       [ 0.,  0.,  0., ..., 30., 11.,  0.]])

### Splitting data

In [115]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature scaling

In [116]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#### DTC

In [125]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [126]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 15.24 %
Standard Deviation: 0.09 %


#### RF

In [127]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [128]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 15.15 %
Standard Deviation: 0.10 %


#### Log Reg

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0, max_iter=500)
classifier.fit(X_train, y_train)

#### SVC

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0, max_iter=100)
classifier.fit(X_train, y_train)

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))